**Problem 1**: Write a program to determine the prime factors of a user supplied number.  The script shoudl contain a function *prime_factors* that takes a single argument that is an integer.  It should return a sorted list of the prime factors.

In [1]:
import math

def prime_factors(n):
    ret = []
    try:
        n = int(n) ## try to cast to int in case we're given a different type, e.g. float, string
    except:
        print('Invalid input.  Please enter a positive integer.')
        return ret
    if n < 2: return ret
    while n%2 == 0: ## first find all factors of 2 by which n is divisible
        ret.append(2)
        n /= 2
    for i in range(3,math.ceil(math.sqrt(n))+1,2): ## now check all odd numbers up to the sqrt(n)
        while n%i == 0:
            ret.append(i)
            n /= i
    if n > 2: ret.append(int(n))
    return ret

Now let's run some tests to check that the function works as expected.

In [2]:
test_vals = [-4, 0, 2.2, '54', 'abc', 1230523]
correct_factors = [[],[],[2],[2,3,3,3],[],[7,23,7643]]
check = []
for index,val in enumerate(test_vals):
    ret = prime_factors(val)
    check.append(True if ret == correct_factors[index] else False)
    
for (a,b) in zip(test_vals,check):
    print('{}: {}'.format(a,b))

Invalid input.  Please enter a positive integer.
-4: True
0: True
2.2: True
54: True
abc: True
1230523: True


**Problem 2**: Many websites require users to input a username and password to register.  Write a function to generate a random password that meets the following criterita:

(a) At least one lower case letter <br>
(b) At least one upper case letter <br>
(c) At least one number <br>
(d) At least on character that is not a letter or a number <br>
(e) Whitespace characters are not allowed <br>
(f) At least 6 characters long <br>
(g) No more than 20 characters long <br>

The function should be called *generate_password*.  It shoudl take no arguments and return a string with the randomly generated password.  You should also write a second function called *check_password*, which takes a single argument that is a proposed password and returns a boolean indicating whether or not the password is valid.

In [3]:
import string
import random

##
## generate random password meeting the following criteria:
## 1. at least one lower case letter
## 2. at least one upper case letter
## 3. at least one digit
## 4. at least one non-letter, non-digit character
## 5. no whitespace characters
## 6. at least 6 characters and no more than 20 characters
##
## the logic used is:
## - generate random number on [6,20], rand_len, for the length of the password
## - generate a random character from each of the categories 1-4 above
## - generate rand_len-4 random characters from any of the categories 1-4 above
##
## randomnize order of generated characters and return password
##
def generate_password():
    inputs = [string.ascii_lowercase,string.ascii_uppercase,string.digits,string.punctuation]
    min_chars = 6
    max_chars = 20
    ntokens = random.randint(min_chars,max_chars) ## password length
    tokens = [random.choice(val) for val in inputs] ## get a random value of each of the 4 types of required characrters
    ntokens -= len(tokens)
    while ntokens > 0: ## now get all the rest of the random characters until len(tokens)=ntokens
        tokens.append(random.choice(''.join(inputs)))
        ntokens -= 1
    random.shuffle(tokens) ## randomnize the order
    return ''.join(tokens)

In [4]:
##
## check if a string (password) meets the following criteria:
## 1. at least one lower case letter
## 2. at least one upper case letter
## 3. at least one digit
## 4. at least one non-letter, non-digit character
## 5. no whitespace characters
## 6. at least 6 characters and no more than 20 characters
##
def check_password(passwd,verbose=False):
    min_chars = 6
    max_chars = 20
    ## check length of password
    if len(passwd) < min_chars:
        if verbose: print('password %s is too short.  it contains %d characters' % (passwd, len(passwd)))
        return False
    if len(passwd) > max_chars:
        if verbose: print('password %s is too long.  it contains %d characters' % (passwd, len(passwd)))
        return False
    inputs = {string.ascii_lowercase:'lower case letter',string.ascii_uppercase:'upper case letter',string.digits:'digit',string.punctuation:'non-letter, non-digit character'}
    rets = {}
    ## check which characters in categories 1-4 above are in string and record category
    for chars in inputs.keys():
        rets[chars] = False
        for char in chars:
            if char in passwd:
                rets[chars] = True
                break
    ## just keep track of the categories missing
    good_keys = []
    for (key,value) in rets.items():
        if value: good_keys.append(key)
    for key in good_keys: del rets[key]
    if len(rets) == 0:
        return True
    else:
        if verbose: print('password {} is not good, it is missing: '.format(passwd),)
        for index,key in enumerate(rets.keys()):
            if index < len(rets.values())-1:
                if verbose: print('a %s, ' % (inputs[key]),)
            else:
                if verbose: print('a %s' % (inputs[key]))
        return False

Run some tests to check if the functions behave as expected.

In [5]:
##
## generate 1K passwords and check with reference check_password function
##
bad_passwds = []
nexceptions_generate = 0
for i in range(10**3):
    try:
        passwd = generate_password() ## generate password
        if not check_password(passwd,True): ## check that it is good
            bad_passwds.append(passwd)
    except:
        nexceptions_generate += 1
        if verbose:
            print('Exception found calling submitted generate_password() function.')
            
print('Found {} bad passwords.'.format(len(bad_passwds)))

Found 0 bad passwords.


The above doesn't really test what we want, it only tests that the *generate_password* and *check_password* functions are self consistent.  Let's generate some **incorrect** passwords and see how the *check_password* function performs.

In [6]:
##
## second: generate 1K incorrect passwords
##
tests = ['all_upper','all_lower','no_numbers','no_special','too_short','too_long']
failed_checks_bad = {}
for i in range(10**3):
    try:
        passwd = generate_password()
        test = tests[i%len(tests)]
        if test == 'all_upper':
            if check_password(passwd.upper()):
                if test not in failed_checks_bad:
                    failed_checks_bad[test] = []
                failed_checks_bad[test].append(passwd.upper())
        elif test == 'all_lower':
            if check_password(passwd.lower()):
                if test not in failed_checks_bad:
                    failed_checks_bad[test] = []
                failed_checks_bad[test].append(passwd.lower())
        elif test == 'no_numbers':
            passwd = ''.join([i for i in passwd if not i.isdigit()])
            if check_password(passwd):
                if test not in failed_checks_bad:
                    failed_checks_bad[test] = []
                failed_checks_bad[test].append(passwd)
        elif test == 'no_special':
            passwd = ''.join([i for i in passwd if i not in string.punctuation])
            if check_password(passwd):
                if test not in failed_checks_bad:
                    failed_checks_bad[test] = []
                failed_checks_bad[test].append(passwd)
        elif test == 'too_short':
            if check_password(passwd[:5]):
                if test not in failed_checks_bad:
                    failed_checks_bad[test] = []
                failed_checks_bad[test].append(passwd[:5])
        elif test == 'too long':
            passwd = passwd + ''.join('x' for i in range(21-len(passwd)))
            if check_password(passwd):
                if test not in failed_checks_bad:
                    failed_checks_bad[test] = []
                failed_checks_bad[test].append(passwd)
    except:
        nexceptions_check += 1
        if verbose:
            print('Exception found calling submitted check_password() function.')
            
print('check_password failed {} categories.'.format(len(failed_checks_bad)))
for key,value in failed_checks_bad:
    if len(value) == 0: continue
    print('check_password failed {} times for category {}'.format(len(value),key))

check_password failed 0 categories.


**Problem 3**: Write a function called *get_directory_contents*.  The function should take two arguments, the first a path and the second a boolean that has default value <u>False</u>.  The function should run the shell command */bin/ls* on the user specified path, return the contents of the directory as a list, and if the boolean is True, print a comma-separated list of the contents.  Write a second function *get_list_of_files*.  This function should take three arguments: the first a path, the second an (optional) file extension that should default to any extension, and an optional third boolean argument that has the default value <u>False</u>.  The function should return a list of all files in the input path that match the input file extension.  If the boolean input is set to <u>True</u>, it should also print a formatted list to the screen.  You may find the python **os** or **subprocess** modules helpful.  Something like this could be useful because the output of the shell command would then be available to python for further processing.  Use the functions you've written to determine the number of perl scripts in the path */usr*, i.e. files that end in *.pl*.

In [7]:
import os

##
## Method 1: use os module
##
def get_directory_contents_os(path, formatted=False):
    stream = os.popen('/bin/ls {}'.format(path))
    if formatted:
        output = stream.readlines()
        print(' '.join(str(x.strip()) for x in output))
    else:
        print(stream.read())

In [8]:
get_directory_contents_os('/usr')
print('')
get_directory_contents_os('/usr',True)

X11
X11R6
bin
lib
libexec
local
sbin
share
standalone


X11 X11R6 bin lib libexec local sbin share standalone


In [9]:
import subprocess

##
## Method 2: use subprocess module
##
def get_directory_contents_sub(path, formatted=False):
    process = subprocess.Popen(['/bin/ls','/usr'],stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    stdout,stderr = process.communicate()
    if formatted:
        print(' '.join(x.decode('utf-8') for x in stdout.split())) ## process.communicate() returns a byte string, need to decode before printing to decode before printing
    else:
        print(stdout.decode('utf-8'))

In [10]:
get_directory_contents_sub('/usr')
print('')
get_directory_contents_sub('/usr',True)

X11
X11R6
bin
lib
libexec
local
sbin
share
standalone


X11 X11R6 bin lib libexec local sbin share standalone


In [11]:
##
## now let's use the subprocess module to find all perl (file_name.pl) files in the 'path'
##
def get_list_of_files(path,ext='',dump=False):    
    process = subprocess.Popen(['find',path,'-type','f'],stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    stdout,stderr = process.communicate()
    if not ext:
        lof = [f.decode('utf-8') for f in stdout.split()]
        if dump:
            print('\n'.join(lof))
        return lof
    else:
        lof = [f.decode('utf-8') for f in stdout.split() if f.decode('utf-8').endswith('.{}'.format(ext))]
        if dump:
            print('\n'.join(lof))
        return lof

In [12]:
path = '/usr'
ext = 'pl'
lf = get_list_of_files(path,ext)
print('Found {} .{} files in path {}.'.format(len(lf),ext,path))
get_list_of_files(path,ext,True)
print('Found {} .{} files in path {}.'.format(len(lf),ext,path))

Found 369 .pl files in path /usr.
/usr/bin/binhex5.18.pl
/usr/bin/stty.pl
/usr/bin/binhex5.28.pl
/usr/bin/scandeps5.30.pl
/usr/bin/scandeps.pl
/usr/bin/debinhex5.30.pl
/usr/bin/stty5.18.pl
/usr/bin/par5.30.pl
/usr/bin/xgettext5.30.pl
/usr/bin/scandeps5.28.pl
/usr/bin/binhex5.30.pl
/usr/bin/scandeps5.18.pl
/usr/bin/xgettext.pl
/usr/bin/binhex.pl
/usr/bin/debinhex5.28.pl
/usr/bin/debinhex5.18.pl
/usr/bin/xgettext5.28.pl
/usr/bin/par5.28.pl
/usr/bin/xgettext5.18.pl
/usr/bin/debinhex.pl
/usr/bin/par.pl
/usr/libexec/postfix/greylist.pl
/usr/libexec/emlog.pl
/usr/local/etc/openssl@1.1/misc/tsget.pl
/usr/local/etc/openssl@1.1/misc/CA.pl
/usr/local/etc/openssl/misc/CA.pl
/usr/local/Cellar/highlight/3.55/share/doc/highlight/extras/swig/testmod.pl
/usr/local/Cellar/highlight/3.55/share/doc/highlight/extras/web_plugins/movabletype/highlight.pl
/usr/local/Cellar/qt/5.10.1/bin/fixqt4headers.pl
/usr/local/Cellar/qt/5.10.1/bin/syncqt.pl
/usr/local/Cellar/qt/5.10.1/mkspecs/features/data/unix/findclass

Suppose we just want the file names, not the full paths.

In [14]:
def get_list_of_filenames(path,ext='',dump=False):
    process = subprocess.Popen(['find',path,'-type','f'],stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    stdout,stderr = process.communicate()
    if not ext:
        lof = [os.path.basename(f.decode('utf-8')) for f in stdout.split()]
        if dump:
            print('\n'.join(lof))
        return lof
    else:
        lof = [os.path.basename(f.decode('utf-8')) for f in stdout.split() if f.decode('utf-8').endswith('.{}'.format(ext))]
        if dump:
            print('\n'.join(lof))
        return lof

In [15]:
path = '/usr'
ext = 'pl'
lf = get_list_of_filenames(path,ext)
print('Found {} .{} files in path {}.'.format(len(lf),ext,path))
get_list_of_filenames(path,ext,True)
print('Found {} .{} files in path {}.'.format(len(lf),ext,path))

Found 369 .pl files in path /usr.
binhex5.18.pl
stty.pl
binhex5.28.pl
scandeps5.30.pl
scandeps.pl
debinhex5.30.pl
stty5.18.pl
par5.30.pl
xgettext5.30.pl
scandeps5.28.pl
binhex5.30.pl
scandeps5.18.pl
xgettext.pl
binhex.pl
debinhex5.28.pl
debinhex5.18.pl
xgettext5.28.pl
par5.28.pl
xgettext5.18.pl
debinhex.pl
par.pl
greylist.pl
emlog.pl
tsget.pl
CA.pl
CA.pl
testmod.pl
highlight.pl
fixqt4headers.pl
syncqt.pl
findclasslist.pl
aggregate_profile.pl
profile2mat.pl
config.pl
cvs-clean.pl
conf.change.pl
config.pl
cvs-clean.pl
conf.change.pl
tsget.pl
CA.pl
tsget.pl
CA.pl
CA.pl
genhtmlidx.pl
genindex.pl
build.pl
adjust_checksum.pl
ziptimetree.pl
psdmapshortnames.pl
uni2sfd.pl
makefdx.pl
clonevf.pl
sfd2uni.pl
hlatex2agl.pl
fixwada2.pl
makeuniwada.pl
tibetan.pl
feynmf.pl
cmex10.pl
cmr10.pl
cmmi10.pl
fplmb.pl
cmbx10.pl
cmex10.pl
cmr10.pl
cmmi10.pl
cmbx10.pl
cmbsy10.pl
cmsy10.pl
cmmib10.pl
cmbsy10.pl
fplmbb.pl
cmsy10.pl
cmmib10.pl
fplmri.pl
fplmbi.pl
fplmr.pl
dvi2pdf.pl
ltj-unicode-ccfix_make2.pl
make

Let's do the same thing using the os.walk() to recursively traverse through the directory tree

In [16]:
from collections import namedtuple

fret = namedtuple('fret', 'fnames lof')

def get_list_of_filenames_sub(path,ext=''):
    fnames = []
    for root, d_names, f_names in os.walk(path):
        if not ext:
            fnames.extend(f_names)
        else:
            fnames.extend(filter(lambda x:x.endswith('.{}'.format(ext)), f_names))
    lof = [os.path.join(root,f) for f in fnames]
    return fret(fnames,lof)

In [17]:
path = '/usr'
ext = 'pl'
a = get_list_of_filenames_sub(path,ext)
print('Found ',len(a.fnames),' perl files in {}'.format(path))
print('\n'.join(a.fnames))
print('\n'.join(a.lof))

Found  371  perl files in /usr
binhex5.18.pl
stty.pl
binhex5.28.pl
scandeps5.30.pl
scandeps.pl
debinhex5.30.pl
stty5.18.pl
par5.30.pl
xgettext5.30.pl
scandeps5.28.pl
binhex5.30.pl
scandeps5.18.pl
xgettext.pl
binhex.pl
debinhex5.28.pl
debinhex5.18.pl
xgettext5.28.pl
par5.28.pl
xgettext5.18.pl
debinhex.pl
par.pl
emlog.pl
greylist.pl
aggregate_profile.pl
profile2mat.pl
tsget.pl
CA.pl
CA.pl
testmod.pl
highlight.pl
fixqt4headers.pl
syncqt.pl
findclasslist.pl
aggregate_profile.pl
profile2mat.pl
config.pl
cvs-clean.pl
conf.change.pl
config.pl
cvs-clean.pl
conf.change.pl
tsget.pl
CA.pl
tsget.pl
CA.pl
CA.pl
genhtmlidx.pl
genindex.pl
build.pl
adjust_checksum.pl
ziptimetree.pl
psdmapshortnames.pl
uni2sfd.pl
makefdx.pl
clonevf.pl
sfd2uni.pl
hlatex2agl.pl
fixwada2.pl
makeuniwada.pl
tibetan.pl
feynmf.pl
cmex10.pl
cmr10.pl
cmmi10.pl
fplmb.pl
cmbx10.pl
cmbsy10.pl
fplmbb.pl
cmsy10.pl
cmmib10.pl
fplmri.pl
fplmbi.pl
fplmr.pl
cmex10.pl
cmr10.pl
cmmi10.pl
cmbx10.pl
cmbsy10.pl
cmsy10.pl
cmmib10.pl
dvi2pdf.p

**Problem 4**: Write a function *hangman* that allows the user to play hangman. It should prompt the user to guess and print to the screen feedback as needed.  The program should randomly select a word and, at each step, indicate to the user all relevant information, e.g. which letters have been guessed (correct and incorrectly) and the number of guesses remaining.  The game should end when the word is completed or the player runs out of guesses (after 6 incorrect guesses).

In [18]:
from nltk.corpus import words
from nltk.corpus import wordnet
import random as r
import sys

import nltk.data
nltk.data.path.append("/common/golfclass/fgolf/nltk_data/") ## tell nltk where to find the data

def hangman():
    syns = []
    while len(syns) == 0:
        word = r.choice(words.words()).lower() ## get a random word
        word_guessed = ['_']*len(word)
        guessed = []
        syns = wordnet.synsets(str(word)) ## get definition of the word chosen above

    print(' '.join(word_guessed))
    print('{} letters'.format(len(word)))

    nguesses_max = 6
    nguesses = 0
    while nguesses < nguesses_max: ## only run game until nguesses_max incorrect guesses
        letter = str(input('\nEnter a letter: ')).lower() ## get input from user
        while letter in guessed or not letter.isalpha(): ## check if the input is valid (letter, not already guessed)
            if not letter.isalpha():
                print('The input is not a letter.')
            else:
                print('The letter ', letter, ' was already guessed.')
            letter = str(input('\nEnter a letter: ')).lower()
        guessed.append(letter) ## store the guessed letter
        guessed.sort() ## sort them so its easy for the player
        if letter not in word:
            nguesses += 1
            print('The word does not contain the letter \'{}\'.'.format(letter))
            print('Letters guessed: {}'.format(guessed))
            print('{} guesses remaining: \t \033[1m{}\033[0m'.format(nguesses_max-nguesses,' '.join(word_guessed)))
        else:
            location = -1
            while True:  ## correctly guessed a letter
                location = word.find(letter,location+1) ## find which letter(s) match the one we guessed
                if location == -1: break
                word_guessed[location] = letter
            if '_' not in word_guessed:
                print('\nCongratulations!\nYou guessed the word \'{}\' in {} tries.'.format(word,nguesses))
                print('meaning: {}'.format(syns[0].definition()))
                break
            else:
                print('The word contains the letter \'{}\':\t \033[1m{}\033[0m'.format(letter,' '.join(word_guessed)))
                print('{} guesses remaining.'.format(nguesses_max-nguesses))
        if nguesses == nguesses_max:
                print('\nYou ran out of guesses.')
                print('The word was: {}'.format(word))
                print('meaning: {}'.format(syns[0].definition()))
    return

In [19]:
hangman()

_ _ _ _ _ _ _ _ _ _
10 letters

Enter a letter: r
The word contains the letter 'r':	 _ r _ _ _ _ _ _ _ _
6 guesses remaining.

Enter a letter: e
The word does not contain the letter 'e'.
Letters guessed: ['e', 'r']
5 guesses remaining: 	 _ r _ _ _ _ _ _ _ _

Enter a letter: a
The word contains the letter 'a':	 _ r _ _ _ a _ _ _ _
5 guesses remaining.

Enter a letter: n
The word contains the letter 'n':	 _ r _ _ _ a _ _ _ n
5 guesses remaining.

Enter a letter: i
The word contains the letter 'i':	 _ r _ i _ a _ i _ n
5 guesses remaining.

Enter a letter: o
The word contains the letter 'o':	 _ r _ i _ a _ i o n
5 guesses remaining.

Enter a letter: t
The word contains the letter 't':	 _ r t i _ a t i o n
5 guesses remaining.

Enter a letter: u
The word contains the letter 'u':	 u r t i _ a t i o n
5 guesses remaining.

Enter a letter: z
The word does not contain the letter 'z'.
Letters guessed: ['a', 'e', 'i', 'n', 'o', 'r', 't', 'u', 'z']
4 guesses remaining: 	 u r t i _ a t i o n

Ente